# Preprocessing B4B data

In [ ]:
import pandas as pd
import numpy as np
import pylab as plt

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

import sys
sys.path.append('../data/')
sys.path.append('../view/')

%load_ext autoreload

%matplotlib inline
%matplotlib widget

from preprocessor import Preprocessor
from plotter import Plot

### Load Measured Data from parquet file

In [ ]:
%%time
# Prerequisite: for this example to work, you need to have the b4b_raw_properties.parquet, located e.g. in the ../data/ folder.
# One way to get this is to run B4BExtractionBackup.ipynb first
df_prop = pd.read_parquet('../data/b4b_raw_properties.parquet', engine='pyarrow')

#sorting the DataFrame index is needed to get good performance on certain filters
#this guarding code to check whether DataFramews are properly sorted
if not df_prop.index.is_monotonic_increasing:
    print('df needed index sorting')
    df_prop = df_prop.sort_index()  

In [ ]:
df_prop.index.unique(level='id').values

In [ ]:
df_prop.index.unique(level='source').values

In [ ]:
df_prop

In [ ]:
df_prop.info()

## Example: processing temp_in__degC

In [ ]:
#if this plot does not show up at initial run, run the cell again (something fishy with interactive plotting of DataFrame.plot.hist())
%matplotlib widget
df_prop.temp_in__degC.plot.hist(bins=50, alpha=0.5)

## Example: using the min-max filter
Filtering out extreme, highly unlikely temperature measurements

In [ ]:
Preprocessor.filter_min_max(df_prop, 'temp_in__degC', min=5, max=40)

In [ ]:
df_prop.info()

In [ ]:
df_prop.describe()

In [ ]:
#if this plot does not show up at initial run, run the cell again (something fishy with interactive plotting of DataFrame.plot.hist())
%matplotlib widget
df_prop.temp_in__degC.plot.hist(bins=50, alpha=0.5)

## Example: using the static outlier filter, per id
Filtering out extreme temperatures based on mean and standard deviation per room

In [ ]:
Preprocessor.filter_static_outliers(df_prop, 'temp_in__degC', n_sigma=3.0, per_id=True)

In [ ]:
#if this plot does not show up at initial run, run the cell again (something fishy with interactive plotting of DataFrame.plot.hist())
%matplotlib widget
df_prop.temp_in__degC.plot.hist(bins=50, alpha=0.5)

In [ ]:
df_prop.info()

In [ ]:
df_prop.describe()

## Example: interpolating all properties


In [ ]:
%%time 
%autoreload 2

property_types = {
    'temp_in__degC' : 'float32',
    'co2__ppm' : 'float32',
    'rel_humidity__0' : 'float32',
    'valve_frac__0' : 'float32',
    'door_open__bool': 'Int8',
    'window_open__bool': 'Int8',
    'occupancy__bool': 'Int8',
    'occupancy__p' : 'Int8'
}

df_interpolated = Preprocessor.interpolate_time(df_prop,
                                        property_dict = property_types,
                                        upsample__min = 5,
                                        interpolate__min = 15,
                                        limit__min = 90,
                                        inplace=False
                                       )

In [ ]:
df_interpolated

In [ ]:
df_interpolated.info()

## Converting raw properties dataframe to preprocessed dataframe


In [ ]:
df_prep = Preprocessor.unstack_prop(df_interpolated)

In [ ]:
df_prep

In [ ]:
df_prep.info()

## Plotting results

In [ ]:
%autoreload 2
units_to_mathtext = property_types = {
    'degC' : r'$°C$',
    'ppm' : r'$ppm$',
    '0' : r'$[-]$',
    'bool': r'$0 = False; 1 = True$',
    'p' : r'$persons$'
}

In [ ]:
%%time
#Plot all properties with a single unit for a single id
Plot.dataframe_properties_plot(df_prop.loc[[999169]][[prop for prop in df_prop.columns.values if prop.split('__')[-1] == 'ppm']], units_to_mathtext)

In [ ]:
%%time
#Plot all preprocessed properties with a single unit for a single id
Plot.dataframe_preprocessed_plot(df_prep.loc[[999169]][[prop for prop in df_prep.columns.values if prop.split('__')[-1] == 'ppm']], units_to_mathtext)

In [ ]:
%%time
#Plot all properties for a single id
Plot.dataframe_preprocessed_plot(df_prep.loc[[999169]], units_to_mathtext)

In [ ]:
%%time
#Plot all properties for a a list of ids
Plot.dataframe_preprocessed_plot(df_prep.loc[[999169, 917810]], units_to_mathtext)

In [ ]:
%%time
#Plot all properties from all sources for all ids WARNING: MAY TAKE LONG!
Plot.dataframe_preprocessed_plot(df_prep, units_to_mathtext)